<a href="https://colab.research.google.com/github/Exabyte-io/api-examples/blob/dev/other/material/import_material_from_jarvis_db_entry.ipynb" target="_parent">
<img alt="Open in Google Colab" src="https://user-images.githubusercontent.com/20477508/128780728-491fea90-9b23-495f-a091-11681150db37.jpeg" width="150" border="0">
</a>

## Install Packages
First, install `express-py` which includes `jarvis-tools` as dependency.

In [ ]:
!pip install express-py==2024.1.25.post7

## Get Materials Data From JARVIS
Then, let's get the dataset containing 2D materials from JARVIS and wrap it into a pandas dataframe.

In [ ]:
import pandas as pd
from jarvis.db.figshare import data, get_jid_data

dft_2d=data('dft_2d')
df = pd.DataFrame(dft_2d)

## Extract Structure and Convert to ESSE
Next, we extract an entry from the JARVIS dataset and convert it into ESSE format ready to be uploaded to Mat3ra.com.

In [ ]:
import json
from express import ExPrESS

## Note: JVASP-670 is an entry for MoTe2
jarvis_db_entry = json.dumps(json.loads(df[(df['jid']=='JVASP-670')].to_json(orient='records'))[0])

kwargs = {
    "structure_string": jarvis_db_entry,
    "cell_type": "original",
    "structure_format": "jarvis-db-entry"
}

handler = ExPrESS("structure", **kwargs)
data = handler.property("material", **kwargs)

# To preview resulting JSON data
print(json.dumps(data, indent=4))

## Setup the API credentials
Finally, we can upload the material to Mat3ra.com using the REST API: follow the explanation in another example notebook [here](../../examples/material/create_material.ipynb). One can replace the content of the `CONFIG` variable with the JSON data above.

In [ ]:
# @title Authorization Form
ACCOUNT_ID = ""  # @param {type:"string"}
AUTH_TOKEN = ""  # @param {type:"string"}
MATERIALS_PROJECT_API_KEY = "MATERIALS_PROJECT_API_KEY"  # @param {type:"string"}
ORGANIZATION_ID = ""


import os

if "COLAB_JUPYTER_IP" in os.environ:
    os.environ.update(
        dict(
            ACCOUNT_ID=ACCOUNT_ID,
            AUTH_TOKEN=AUTH_TOKEN,
            MATERIALS_PROJECT_API_KEY=MATERIALS_PROJECT_API_KEY,
            ORGANIZATION_ID=ORGANIZATION_ID,
        )
    )

    !GIT_BRANCH="dev"; export GIT_BRANCH; curl -s "https://raw.githubusercontent.com/Exabyte-io/api-examples/${GIT_BRANCH}/scripts/env.sh" | bash

## Initialize the API Endpoints

In [ ]:
from utils.settings import ENDPOINT_ARGS, ACCOUNT_ID
from utils.generic import wait_for_jobs_to_finish, get_property_by_subworkflow_and_unit_indicies, dataframe_to_html, display_JSON

import pandas as pd

# Relevant functions from the API client
from exabyte_api_client.endpoints.jobs import JobEndpoints
from exabyte_api_client.endpoints.projects import ProjectEndpoints
from exabyte_api_client.endpoints.materials import MaterialEndpoints
from exabyte_api_client.endpoints.bank_workflows import BankWorkflowEndpoints
from exabyte_api_client.endpoints.properties import PropertiesEndpoints

job_endpoints = JobEndpoints(*ENDPOINT_ARGS)
project_endpoints = ProjectEndpoints(*ENDPOINT_ARGS)
material_endpoints = MaterialEndpoints(*ENDPOINT_ARGS)
property_endpoints = PropertiesEndpoints(*ENDPOINT_ARGS)
bank_workflow_endpoints = BankWorkflowEndpoints(*ENDPOINT_ARGS)

## Create and Run a Job

Setup entities first:

In [ ]:
default_material = material_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]
default_workflow = workflow_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]

material_id = default_material["_id"]
workflow_id = default_workflow["_id"]
owner_id = default_material["owner"]["_id"]

Then create Job config

In [ ]:
config = {
    "owner": {"_id": owner_id},
    "_material": {"_id": material_id},
    "workflow": {"_id": workflow_id},
    "name": "Test Job",
}

Then create a job

In [ ]:
job = job_endpoints.create(config)
job_endpoints.submit(job["_id"])

and wait for it to finish

In [ ]:
from utils.generic import wait_for_jobs_to_finish, get_property_by_subworkflow_and_unit_indicies
job_id = job["_id"]
wait_for_jobs_to_finish(job_endpoints, [job_id])

Then get job files:

In [ ]:
files = job_endpoints.list_files(job_id)
paths = [file["key"] for file in files]
for path in paths:
    if "outdir" not in path:
        print(path)

for file in files:
    if file["name"] == "pw_scf.out":
        output_file_metadata = file

import urllib

server_response = urllib.request.urlopen(output_file_metadata["signedUrl"])
output_file = server_response.read().decode(encoding="UTF-8")

with open(output_file_metadata["name"], "w") as file_descriptor:
    file_descriptor.write(output_file)

and get job properties using the index of a subworkflow in workflow and the index of unit in subworkflow (0, 0) below

In [ ]:
pressure = get_property_by_subworkflow_and_unit_indicies(property_endpoints, "pressure", job, 0, 0)
print(pressure["data"]["value"])